In [1]:
import re
from konlpy.tag import Okt
from nltk import FreqDist
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Embedding, SimpleRNN, RNN, LSTM, GRU, Input, Dense, Bidirectional, Masking, GlobalMaxPooling1D, Conv1D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import string
from scipy.stats import mode

In [15]:
df=pd.read_csv('../../datasets/nlp/project/한국어 대화/키워드.csv')

In [16]:
df['Text']=df.Q+' '+df.A

In [17]:
df=df[['Text','S']]

In [18]:
df.columns=['text','label']

In [19]:
df

,text,label
0,시간에 얼마인가요 처음 시간은 원이고 이후 시간은 원씩 추가됩니다,pc방
1,무인발급기 있나요 무인발급기는 카운터 바로 옆쪽에 이용 가능합니다,pc방
2,달 정액권 끊을 수 있나요 네 달에 만 원입니다,pc방
3,정액권 끊다가 정지해도 되나요 네 가능합니다,pc방
4,음식 주문 가능한가요 햄버거랑 핫도그 종류 가능합니다,pc방
...,...,...
29241,일주일에 월화수목금 오면 얼마에요 저희 초등학생 같은 경우에는 만원 만원이요,미술학원
29242,그럼 월수만 할 경우에는 어떻게 해요 그럼 만원 정도에요 중학생이라고 더 받고 그런...,미술학원
29243,전 영역을 다 가르치시는 거에요 네 저학년은 수채화랑 디자인 가르치고 고학년은 수묵...,미술학원
29244,차량 같은 거는 없죠 네 차량은 없어요,미술학원


In [20]:
df.label.unique().shape

(31,)

In [21]:
tk=Okt()

In [22]:
# 불용어 단어 모음
with open('../../datasets/nlp/stopwords_kor.txt') as f:
    stopwords=f.readlines()
def replace_n(text):
    text=text.replace('\n','')
    return text
stopwords=list(map(replace_n,stopwords))

In [23]:
sample_text=df.iloc[0,0]
' '.join([i for i in tk.morphs(sample_text) if i not in stopwords])

'시간 인가요 처음 시간 원 이후 시간 원 씩 추가 됩니다'

In [24]:
from tqdm.notebook import tqdm

In [25]:
for i in tqdm(range(df.shape[0])):
    df.iloc[i,0]=' '.join([i for i in tk.morphs(df.iloc[i,0]) if i not in stopwords])

  0%|          | 0/29246 [00:00<?, ?it/s]

In [26]:
df

,text,label
0,시간 인가요 처음 시간 원 이후 시간 원 씩 추가 됩니다,pc방
1,무인 발급 있나요 무인 발급 기는 카운터 옆쪽 이용 가능합니다,pc방
2,달 정액 권 끊을 있나요 달 원,pc방
3,정액 권 끊다가 정지 해도 되나요 가능합니다,pc방
4,음식 주문 가능한가요 햄버거 핫도그 종류 가능합니다,pc방
...,...,...
29241,일주일 월화 목금 오면 초등학생 경우 만원 만원,미술학원
29242,수만 경우 만원 중학생 받고 그런 없어요 학생 크니깐 본인 결정 좋을,미술학원
29243,전 영역 가르치시는 저학년 수채화 디자인 가르치고 고학년 수묵화 특강 가르치고 있어...,미술학원
29244,차량 없죠 차량 없어요,미술학원


In [27]:
tk=Tokenizer()

In [28]:
tk.fit_on_texts(df.text)

In [29]:
word_counts=pd.DataFrame(tk.word_counts.items())

In [30]:
word_counts=word_counts.sort_values(by=1,ascending=False)

In [31]:
NUM_WORDS=word_counts[word_counts[1]>30].shape[0]
OOV='<OOV>'

In [32]:
tk=Tokenizer()
tk.fit_on_texts(df.text)
word_counts=pd.DataFrame(tk.word_counts.items())
word_counts=word_counts.sort_values(by=1,ascending=False)
NUM_WORDS=word_counts[word_counts[1]>30].shape[0]
NUM_WORDS

1206

In [33]:
tk=Tokenizer(oov_token=OOV,num_words=NUM_WORDS)

In [34]:
tk.fit_on_texts(df.text)

In [35]:
sequences=tk.texts_to_sequences(df.text)

In [36]:
tk=Tokenizer(oov_token=OOV,num_words=NUM_WORDS)
tk.fit_on_texts(df.text)
sequences=tk.texts_to_sequences(df.text)
seqTextLen=[len(text) for text in sequences]
print(f'최소 토큰수 : {min(seqTextLen)} \n최대 토큰수 : {max(seqTextLen)} \n평균토큰수 : {sum(seqTextLen)/len(seqTextLen)} \n토큰 중앙값 : {sorted(seqTextLen)[int(len(seqTextLen)/2)]}')

최소 토큰수 : 0 
최대 토큰수 : 49 
평균토큰수 : 7.503897968953019 
토큰 중앙값 : 7


In [37]:
TEXT_LENGTH=8

In [38]:
textData=pad_sequences(sequences,maxlen=TEXT_LENGTH)

In [189]:
textData.shape

(29246, 8)

In [40]:
le = LabelEncoder()
textLabel = le.fit_transform(df.label)

In [191]:
textLabel.shape

(29246,)

In [41]:
X_train,X_test,y_train,y_test=train_test_split(textData,textLabel,test_size=0.2,stratify=textLabel)

In [42]:
X_train=to_categorical(X_train)
X_test=to_categorical(X_test)
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [43]:
TEXT_LENGTH=8
textData=pad_sequences(sequences,maxlen=TEXT_LENGTH)
le = LabelEncoder()
textLabel = le.fit_transform(df.label)
X_train,X_test,y_train,y_test=train_test_split(textData,textLabel,test_size=0.2,stratify=textLabel)
X_train=to_categorical(X_train)
X_test=to_categorical(X_test)
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
X_train.shape, y_train.shape

((23396, 8, 1206), (23396, 31))

In [195]:
model = Sequential()
model.add(LSTM(1024,input_shape=(X_train.shape[1:]),return_sequences=True))
model.add(LSTM(512)) 
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(31, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(X_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(X_test, y_test))

Epoch 1/30
183/183 [==============================] - ETA: 0s - loss: 2.5837 - acc: 0.2746
Epoch 1: val_acc improved from -inf to 0.43487, saving model to best_model.h5
183/183 [==============================] - 96s 504ms/step - loss: 2.5837 - acc: 0.2746 - val_loss: 2.0297 - val_acc: 0.4349
Epoch 2/30
183/183 [==============================] - ETA: 0s - loss: 1.8124 - acc: 0.4835
Epoch 2: val_acc improved from 0.43487 to 0.48256, saving model to best_model.h5
183/183 [==============================] - 134s 733ms/step - loss: 1.8124 - acc: 0.4835 - val_loss: 1.8332 - val_acc: 0.4826
Epoch 3/30
183/183 [==============================] - ETA: 0s - loss: 1.5833 - acc: 0.5402
Epoch 3: val_acc improved from 0.48256 to 0.53128, saving model to best_model.h5
183/183 [==============================] - 127s 693ms/step - loss: 1.5833 - acc: 0.5402 - val_loss: 1.6539 - val_acc: 0.5313
Epoch 4/30
183/183 [==============================] - ETA: 0s - loss: 1.4608 - acc: 0.5697
Epoch 4: val_acc impro

In [5]:
loaded_model = load_model(r'C:\EXAM_python\datasets\nlp\project\best_model.h5')
# print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [47]:
def guess_where(test_text):
    test_text=re.sub('[^가-힣 ]', '',test_text)
    test_text=[i for i in okt.morphs(test_text) if i not in stopwords]
    test_text=np.array(tk.texts_to_sequences(test_text)).reshape(1,-1)
    test_text=pad_sequences(test_text,maxlen=TEXT_LENGTH)
    test_text=to_categorical(test_text,num_classes=NUM_WORDS)
    print(le.inverse_transform((np.argmax(loaded_model.predict(test_text)),)))

In [198]:
le.inverse_transform((range(1,31,1)))

array(['가방', '당구장', '독서실', '떡집', '모텔여관', '미술학원', '미용실', '반찬가게', '배달음식점',
       '부동산', '세탁소', '세탁소 ', '셀프서비스', '신발', '액세서리', '약국', '어학원', '영수학원',
       '옷수선', '의류', '정육점', '제과점', '청과물', '카페', '태권도', '펜션캠핑장', '피아노',
       '호텔', '홀서빙음식점', '화장품'], dtype=object)

In [48]:
test_text='여기 학원비가 얼마에요? 성인반은 13만원입니다 '
guess_where(test_text)

1/1 [==============================] - 1s 591ms/step
['미술학원']


In [49]:
test_text='어떤 스타일로 잘라드릴까요? 알아서 잘라주세요'
guess_where(test_text)

1/1 [==============================] - 0s 46ms/step
['미용실']


In [50]:
test_text='페퍼로니 피자 라지 한판 카드 결제 할게요. 네'
guess_where(test_text)

1/1 [==============================] - 0s 29ms/step
['배달음식점']
